In [1]:
import pandas as pd
from glob import glob
import datetime
import random
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import numpy as np

In [2]:
def cal_map_cat(x,dict_,i):
    if x[i] in dict_:
        return dict_[x[i]]
    else:
        return x[i]

class kv_pair(dict): 
    def __init__(self): 
        self = dict() 
    def add(self, key, value): 
        self[key] = value 


In [3]:
df = pd.read_csv('Toronto_data.csv')
loc = pd.read_csv('Toronto_station_data.csv')

C:\Users\akbarimm\AppData\Local\Temp\ipykernel_26220\2673023240.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Toronto_data.csv')


In [4]:
loc=loc[(loc['station_id'].isin(df['end_station_id'].unique()))&(loc['station_id'].isin(df['end_station_id'].unique()))]

In [5]:
df['started_at']=pd.to_datetime(df['started_at'])
df['ended_at']=pd.to_datetime(df['ended_at'])
df['year']=df['started_at'].apply(lambda x: x.year)
df['hour']=df['started_at'].apply(lambda x: x.hour)

In [6]:
df.columns

Index(['ride_id', 'trip_duration_seconds', 'start_station_id', 'started_at',
       'start_station_name', 'end_station_id', 'ended_at', 'end_station_name',
       'Bike number', 'member_casual', 'Duration', 'End Time_copy',
       'from_station_name', 'weekday', 'is_weekend', 'month', 'year',
       'quarter', 'start_lat', 'start_lng', 'start_station_capacity',
       'end_lat', 'end_lng', 'end_station_capacity', 'started_hour', 'period',
       'hour'],
      dtype='object')

In [8]:
def cal_map_cat(x,dict_,i):
    if x[i] in dict_:
        return dict_[x[i]]
    else:
        return x[i]

class kv_pair(dict): 
    def __init__(self): 
        self = dict() 
    def add(self, key, value): 
        self[key] = value 
dict_weekday={'monday':'Monday','tuesday':'Tuesday','wednesday':'Wednesday','thursday':'Thursday','friday':'Friday','saturday':'Saturday','sunday':'Sunday'}
df['weekday']=df.apply(cal_map_cat, args=(dict_weekday,'weekday',), axis=1)   

In [9]:
t2=pd.DataFrame(columns=['variable'])
for col in ['period','month','dummy','weekday']:
    t1=pd.DataFrame(columns=['variable','sub_variable'])
    for year in sorted(df['year'].unique()):
        t_=df[df['year']==year]
        t_['dummy']=[1 for i in range(len(t_))]
        t__=t_.groupby(['year',col]).count().reset_index()[['year',col,'ride_id']]
        # t__[col]=t__[col].apply(lambda x: col+'_'+str(x))
        t__['variable']=col
        t__=t__.rename(columns={'ride_id':year,col:'sub_variable'})
        t__=t__[['variable','sub_variable',year]]
        t1=pd.merge(t1,t__,on=['variable','sub_variable'],how='outer')
    t2=pd.concat([t2,t1])
# t2.to_csv('stats_rides.csv',index=False)

C:\Users\akbarimm\AppData\Local\Temp\ipykernel_26220\537750361.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_['dummy']=[1 for i in range(len(t_))]
C:\Users\akbarimm\AppData\Local\Temp\ipykernel_26220\537750361.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_['dummy']=[1 for i in range(len(t_))]
C:\Users\akbarimm\AppData\Local\Temp\ipykernel_26220\537750361.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [10]:
start_station_list = list(df['start_station_id'].unique())
end_station_list = list(df["end_station_id"].unique())
all_stations =set(start_station_list+end_station_list)

In [11]:
# "Number of stations"
print(len(all_stations))

575


In [13]:
station_counts_from =pd.DataFrame(df["start_station_id"].value_counts()).sort_index(ascending=True)
station_counts_from.rename(columns={'count':'checkins'}, inplace=True)

station_counts_to= pd.DataFrame(df["end_station_id"].value_counts()).sort_index(ascending=True)
station_counts_to.rename(columns={'count':'checkouts'}, inplace=True)
station_counts = pd.concat([station_counts_from, station_counts_to], axis=1)

station_counts['total'] = station_counts['checkouts'] + station_counts['checkins']

In [14]:
num_=100
station_counts_100 = station_counts.sort_values('total', ascending=False).head(num_)
# station_counts_100 = station_counts.sort_values('total', ascending=False)
stations = list(station_counts_100.index)
df_station = df[(df['start_station_id'].isin(stations)) | (df['end_station_id'].isin(stations))]
total_trip_count = len(df)


In [15]:
Rent_df = df[df['start_station_id'].isin(list(station_counts_100.index))][['started_at','start_station_name','start_station_id','end_station_name','end_station_id','ended_at']]
Return_df = df[df['end_station_id'].isin(list(station_counts_100.index))][['started_at','start_station_name','start_station_id','end_station_name','end_station_id','ended_at']]

In [16]:
dict_period={0:'Early Morning',1:'Early Morning',2:'Early Morning',3:'Early Morning',4:'Early Morning',5:'Early Morning',
         6:'Morning',7:'Morning',8:'Morning',9:'Morning',10:'Morning',11:'Morning',
         12:'Afternoon',13:'Afternoon',14:'Afternoon',15:'Afternoon',16:'Afternoon',17:'Afternoon',
         18:'Night',19:'Night',20:'Night',21:'Night',22:'Night',23:'Night'}

def update_df(Rent_df):
    Rent_df['year']=Rent_df['started_at'].apply(lambda x: x.year)
    Rent_df['hour']=Rent_df['started_at'].apply(lambda x: x.hour)
    Rent_df['month']=Rent_df['started_at'].apply(lambda x: x.month)
    Rent_df['period']=Rent_df['hour'].copy()
    Rent_df['period']=Rent_df.apply(cal_map_cat, args=(dict_period,'period',), axis=1)
    day = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    Rent_df["weekday"] = Rent_df['started_at'].apply(lambda x: pd.Timestamp(x).dayofweek)
    Rent_df["weekday"] = Rent_df["weekday"].apply(lambda x: day[x])
    return Rent_df
Rent_df=update_df(Rent_df)
Return_df=update_df(Return_df)

In [17]:
# Rent_df.to_csv('RentTop_whole.csv',index=False)
# Return_df.to_csv('ReturnTop_whole.csv',index=False)